In [21]:
pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.


In [43]:
import shutil
import os

In [63]:
# Каталог с набором данных
data_dir = '/home/maksim/PycharmProjects/bonus_sys/for_bonus'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 157

In [64]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "100"))
    os.makedirs(os.path.join(dir_name, "invalid"))

In [65]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [66]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "hundred" + str(i) + ".jpg"), 
                    os.path.join(dest_dir, "100"))
        shutil.copy2(os.path.join(source_dir, "invalid" + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "invalid"))

In [67]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)

109
133


In [68]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

# Обучение

In [69]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

In [70]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 90, 45
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 109
# Количество изображений для проверки
nb_validation_samples = 24
# Количество изображений для тестирования
nb_test_samples = 24

In [71]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [72]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [73]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [74]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 218 images belonging to 2 classes.


In [75]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 48 images belonging to 2 classes.


In [76]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 48 images belonging to 2 classes.


In [77]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
6/6 [==============================] - 1s 113ms/step - loss: 0.7123 - accuracy: 0.4792 - val_loss: 0.7347 - val_accuracy: 0.3125
Epoch 2/30
6/6 [==============================] - 0s 55ms/step - loss: 0.7037 - accuracy: 0.4688 - val_loss: 0.6787 - val_accuracy: 0.5625
Epoch 3/30
6/6 [==============================] - 0s 59ms/step - loss: 0.6856 - accuracy: 0.4688 - val_loss: 0.6884 - val_accuracy: 0.2500
Epoch 4/30
6/6 [==============================] - 0s 53ms/step - loss: 0.6405 - accuracy: 0.6111 - val_loss: 0.7320 - val_accuracy: 0.3750
Epoch 5/30
6/6 [==============================] - 0s 57ms/step - loss: 0.6523 - accuracy: 0.6111 - val_loss: 0.6252 - val_accuracy: 0.8750
Epoch 6/30
6/6 [==============================] - 0s 61ms/step - loss: 0.5968 - accuracy: 0.6354 - val_loss: 0.5481 - val_accuracy: 0.9375
Epoch 7/30
6/6 [==============================] - 0s 51ms/step - loss: 0.5350 - accuracy: 0.8000 - val_loss: 0.4649 - val_accuracy: 1.0000
Epoch 8/30
6/6 [==========

In [78]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [79]:
print("%.2f%%" % (scores[1]*100))

87.50%
